In [ ]:
import pandas as pd
import numpy as np

from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
ATAC_n43_set = pd.read_csv("data/ATAC_n43_set_processed_2018-10-09.csv", index_col = 0)
K4_w_NPC_broad_set = pd.read_csv("data/K4_w_NPC_broad_set_processed_2018-10-09.csv", index_col = 0)
ATAC_n43_set.head()
K4_w_NPC_broad_set.head()

In [ ]:
class heatmap():
    def __init__(self, peak_set_file):
        df = pd.read_csv(peak_set_file, index_col = 0)
        self.df_ChIP = {ChIP_exp : df.loc[df.ChIP_exp == ChIP_exp] for ChIP_exp in df.ChIP_exp.unique()}
        self.heat_map_type = {"percentile" : self.make_heat_map_percentile, "pvalue": self.make_heat_map_pvalue}
        
    def make_heat_map_percentile(self, ChIP_exp, metric, threshold, width, length):
        df = self.df_ChIP[ChIP_exp].copy()
        df.loc[:,"binary"] = np.where(df.loc[:, metric] >= threshold, 1, 0)

        
        result = df.pivot(index = "rsID", columns = "Cell_Type", values = "binary")
        fig, ax = plt.subplots(1,1, figsize = (width,length))
        ax.tick_params(labelsize= 10)
        ax = sns.heatmap(result, annot=False, cmap = "viridis", cbar_kws= {"ticks": [0,1]}, linewidths=1)
        return ax
    
    def make_heat_map_pvalue(self, ChIP_exp, metric, threshold, width, length):
        df = self.df_ChIP[ChIP_exp].copy()
        df.loc[:,metric] = np.log(df.loc[:,metric])
        df.loc[:,"binary"] = np.where(df.loc[:,metric] >= threshold, 1, 0)
        
        result = df.pivot(index = "rsID", columns = "Cell_Type", values = "binary")
        fig, ax = plt.subplots(1,1, figsize = (width,length))
        ax.tick_params(labelsize= 10)
        ax = sns.heatmap(result, annot=True, fmt = '1.2E', cmap = "viridis", linewidths=1)
        return ax
        
    def make_heat_map(self, *, heat_map_type, ChIP_exp, metric, threshold, width, length):
        heat_map_func = self.heat_map_type[heat_map_type]
        heat_map_func(ChIP_exp, metric, threshold, width, length)
        
        
# def generate_heatmap(PEAK_SET, ChIP_exp, metric, threshold):
#     if PEAK_SET == "K4_w_NPC_broad_set":
#         df = K4_w_NPC_broad_set
#     else:
#         df = ATAC_n43_set
#     df_ChIP = df.loc[df.ChIP_exp == ChIP_exp]
#     df_ChIP = df_ChIP.dropna(axis = 0)
#     df_ChIP = df_ChIP.dropna(axis = 1)
#     df_ChIP.loc[:,"binary"] = np.where(df_ChIP.loc[:,metric] >= threshold, 1, 0)
#     result = df_ChIP.pivot(index = "rsID", columns = "Cell_Type", values = "binary")
#     fig, ax = plt.subplots(1,1, figsize = (10,30))
#     ax.tick_params(labelsize= 10)
#     ax = sns.heatmap(result, annot=False, cmap = "viridis", cbar_kws= {"ticks": [0,1]}, linewidths=1)
#     return ax


# a = ['H3K27ac', 'H3K4me1', 'E041', 'H3K27ac_roadmap_E041',
#        'H3K27ac_roadmap_E042', 'E042']

# b = ["K4_w_NPC_broad_set","ATAC_n43_set"]


# metric = ['KS_pvalue_percentile', "paired_ttest_pvalue_percentile", "zero_minus_one_percentile_bootstrap", 'mann_whitney_pvalue_percentile']

# threshold = widgets.IntSlider(min=0,max=100,step=1,value=90)

# interact_manual(generate_heatmap, ChIP_exp = a, PEAK_SET = b, threshold = threshold, metric = metric);

In [ ]:
a = ['H3K27ac', 'H3K4me1', 'E041', 'H3K27ac_roadmap_E041',
       'H3K27ac_roadmap_E042', 'E042']
b = ["K4_w_NPC_broad_set","ATAC_n43_set"]
metric_type = {"": "", "percentile":['KS_pvalue_percentile', "paired_ttest_pvalue_percentile", "zero_minus_one_percentile_bootstrap", 'mann_whitney_pvalue_percentile'],\
               "pvalue": ["realdata_KS_pvalue", "realdata_mann_whitney_pvalue"]}

metric = widgets.Dropdown(options = [])
heat_map_type = widgets.Dropdown(options=["","percentile","pvalue"])

def change_heat_map_type(change):
    metric.options = metric_type[change.new]
heat_map_type.observe(change_heat_map_type, "value")

In [ ]:
ATAC_n43_set = heatmap("data/ATAC_n43_set_processed_2018-10-09.csv")
K4_w_NPC_broad_set = heatmap("data/K4_w_NPC_broad_set_processed_2018-10-09.csv")

In [ ]:
interact_manual(ATAC_n43_set.make_heat_map, heat_map_type = heat_map_type,ChIP_exp = a, metric = metric, threshold=90, width = 10, length = 30)

In [ ]:
interact_manual(K4_w_NPC_broad_set.make_heat_map, heat_map_type = heat_map_type,ChIP_exp = a, metric = metric, threshold=90, width = 10, length = 30)